## Two-Stream Radiative Transfer Solver
**Objective**: Solve the two-stream equations for an optically thick layer.

**Key Equations**:

$-\frac{1}{2} \frac{dI_1}{d\tau} = -I_1 + \frac{\omega}{2} [(1+\beta) I_1 + (1-\beta) I_2]$

$\frac{1}{2} \frac{dI_2}{d\tau} = -I_2 + \frac{\omega}{2} [(1-\beta) I_1 + (1+\beta) I_2]$

( $\tau$ ) : Profondeur optique (0 = surface, τ0τ0​ = base)

( $\omega$ ) : Albédo de diffusion simple

( $\beta$ ) : Fraction de rétrodiffusion

**Boundary Conditions**:
- At $ \tau = 0 $: $ I_2(0) = (1-S_e)I_0 + S_i I_1(0) $
- At $ \tau = \tau_0 $: $ I_1(\tau_0) = S_{ib} I_2(\tau_0) $

**Tasks**:
1. Implement the two-stream equations
2. Solve numerically using finite differences or ODE solver
3. Plot $ I_1(\tau) $ and $ I_2(\tau) $

**Challenge**: Add wavelength-dependent absorption $ \kappa(\lambda) $

In [ ]:
from scipy.integrate import solve_ivp
from scipy.optimize import fsolve
import numpy as np
import matplotlib.pyplot as plt

# 1. Définition des équations two-stream (version corrigée)
def two_stream(tau, I, omega, beta, S_e, S_i, I0):
    I1, I2 = I  # I1: montante, I2: descendante
    dI1dtau = 2 * I1 - omega * ((1+beta)*I1 + (1-beta)*I2)
    dI2dtau = -2 * I2 + omega * ((1-beta)*I1 + (1+beta)*I2)
    return [dI1dtau, dI2dtau]  # Conversion explicite en array numpy

# 2. Paramètres physiques (slides 4-6)
omega = 0.8     # Albédo de diffusion simple
beta = 0.3      # Fraction de rétrodiffusion
tau_max = 5     # Profondeur optique totale
I0 = 1.0        # Flux incident normalisé
S_e, S_i, S_ib = 0.1, 0.2, 0.1  # Coefficients de surface

# 3. Fonction de condition aux limites (version stable)
def boundary_condition(I1_guess):
    # Solution initiale avec guess pour I1(0)
    sol = solve_ivp(two_stream, [0, tau_max], 
                    np.array([I1_guess, (1-S_e)*I0 + S_i*I1_guess]),  # y0 doit être 1D
                    args=(omega, beta, S_e, S_i, I0),
                    dense_output=True)
    
    # Vérification condition à tau_max
    I1_final, I2_final = sol.sol(tau_max)
    return I1_final - S_ib * I2_final  # Doit retourner un scalaire!

# 4. Résolution numérique (méthode robuste)
I1_0_initial_guess = 0.5  # Valeur initiale raisonnable
I1_0_solution = fsolve(boundary_condition, I1_0_initial_guess)[0]

# 5. Intégration finale avec la bonne condition initiale
final_solution = solve_ivp(two_stream, [0, tau_max],
                          np.array([I1_0_solution, (1-S_e)*I0 + S_i*I1_0_solution]),
                          args=(omega, beta, S_e, S_i, I0),
                          t_eval=np.linspace(0, tau_max, 100))

# 6. Visualisation professionnelle
plt.figure(figsize=(10, 6))
plt.plot(final_solution.t, final_solution.y[0], 'r-', lw=2, label='$I_1$ (flux montant)')
plt.plot(final_solution.t, final_solution.y[1], 'b--', lw=2, label='$I_2$ (flux descendant)')
plt.xlabel('Profondeur optique $\\tau$', fontsize=12)
plt.ylabel('Intensité normalisée', fontsize=12)
plt.title('Solution des équations two-stream\n(Conditions aux limites: $S_e$=%.1f, $S_i$=%.1f, $S_{ib}$=%.1f)' 
          % (S_e, S_i, S_ib), pad=20)
plt.legend(fontsize=10, framealpha=0.9)
plt.grid(True, alpha=0.3)
plt.show()

TypeError: two_stream() missing 6 required positional arguments: 'I', 'omega', 'beta', 'S_e', 'S_i', and 'I0'